In [1]:
import pandas as pd
import numpy as np
import re
import random

c:\Users\WONJONGHYEON\anaconda3\envs\CP2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 데이터 불러오기 및 분리

In [1062]:
df=pd.read_csv('train.csv').drop(['ID'], axis=1)
trans_aug=pd.read_csv(r'C:\Users\WONJONGHYEON\OneDrive\바탕 화면\cp2-competition\augment_csv\trans_aug.csv')

In [ ]:
df

In [ ]:
trans_aug

In [1064]:
df.drop_duplicates(subset=['문장', 'label'], inplace=True) #문장, 레이블 동시 중복 제거
df.reset_index(drop=True, inplace=True)

df['확실성'][14963]='확실'
df['label'][14963]='사실형-긍정-현재-확실' #맞게 변경

df.drop_duplicates(subset='문장', keep='last', inplace=True) #틀린 레이블을 가진 중복 문장 제거
df.reset_index(drop=True, inplace=True)

In [1065]:
label_table = df['label'].value_counts().to_frame()
label_table.head()

,label
사실형-긍정-과거-확실,7099
사실형-긍정-현재-확실,4735
추론형-긍정-현재-확실,1099
사실형-긍정-미래-확실,647
추론형-긍정-과거-확실,334


In [546]:
aug_label_table = trans_aug['label'].value_counts().to_frame()
aug_label_table.head()

,label
예측형-긍정-미래-불확실,263
예측형-긍정-미래-확실,107
대화형-미정-현재-불확실,47
사실형-미정-미래-확실,42
사실형-미정-미래-불확실,38


In [547]:
labels = label_table[label_table.label>=10].index
aug_labels = aug_label_table[aug_label_table.label>=10].index

In [550]:
idx_lst, aug_idx_lst = [], []

for idx, label in enumerate(df['label']):
    if not label in list(labels):
        idx_lst.append(idx)

for idx, label in enumerate(trans_aug['label']):
    if not label in list(aug_labels):
        aug_idx_lst.append(idx)

In [552]:
df_in=df.drop(idx_lst)
df_in.reset_index(drop=True, inplace=True)

df_out=df.iloc[idx_lst]
df_out.reset_index(drop=True, inplace=True)
df_out=df_out.sample(frac=1)


In [553]:
aug_in=trans_aug.drop(aug_idx_lst)
aug_in.reset_index(drop=True, inplace=True)

aug_out=trans_aug.iloc[aug_idx_lst]
aug_out.reset_index(drop=True, inplace=True)
aug_out=aug_out.sample(frac=1)

In [570]:
from sklearn.model_selection import train_test_split
df_train, df_val, y_t, y_v = train_test_split(df_in[['문장', '유형', '극성', '시제', '확실성']], df_in[['label']], test_size=0.2, stratify=df_in['label'], random_state=1)
aug_train, aug_val, aug_y_t, aug_y_v = train_test_split(aug_in[['문장', '유형', '극성', '시제', '확실성']], aug_in[['label']], test_size=0.2, stratify=aug_in['label'], random_state=1)

In [573]:
aug_y_t.value_counts(normalize=True)

label        
예측형-긍정-미래-불확실    0.375000
예측형-긍정-미래-확실     0.151786
대화형-미정-현재-불확실    0.067857
사실형-미정-미래-확실     0.060714
대화형-미정-미래-불확실    0.053571
사실형-미정-미래-불확실    0.053571
추론형-미정-미래-불확실    0.046429
사실형-미정-현재-확실     0.037500
예측형-긍정-현재-확실     0.032143
예측형-미정-미래-불확실    0.030357
예측형-긍정-과거-확실     0.025000
예측형-긍정-현재-불확실    0.023214
대화형-미정-과거-불확실    0.021429
예측형-긍정-과거-불확실    0.021429
dtype: float64

In [572]:
aug_y_v.value_counts(normalize=True)

label        
예측형-긍정-미래-불확실    0.375887
예측형-긍정-미래-확실     0.156028
대화형-미정-현재-불확실    0.063830
사실형-미정-미래-불확실    0.056738
사실형-미정-미래-확실     0.056738
대화형-미정-미래-불확실    0.049645
추론형-미정-미래-불확실    0.049645
사실형-미정-현재-확실     0.035461
예측형-긍정-현재-확실     0.035461
예측형-긍정-과거-확실     0.028369
예측형-미정-미래-불확실    0.028369
대화형-미정-과거-불확실    0.021277
예측형-긍정-과거-불확실    0.021277
예측형-긍정-현재-불확실    0.021277
dtype: float64

In [555]:
len(df_out), len(aug_out)

(92, 52)

In [556]:
df_train = pd.concat([df_train, df_out[:41].drop(['label'], axis=1)], axis=0)
df_val = pd.concat([df_val, df_out[41:].drop(['label'], axis=1)], axis=0)
aug_train = pd.concat([aug_train, aug_out[:26].drop(['label'], axis=1)], axis=0)
aug_val = pd.concat([aug_val, aug_out[26:].drop(['label'], axis=1)], axis=0)

In [563]:
df_train = pd.concat([df_train, aug_train], axis=0)
df_train.drop_duplicates(inplace=True)
df_val = pd.concat([df_val, aug_val], axis=0)
df_val.drop_duplicates(inplace=True)

In [ ]:
df_train['유형'].value_counts(), df_val['유형'].value_counts()

In [ ]:
df_train['극성'].value_counts(), df_val['극성'].value_counts()

In [ ]:
df_train['시제'].value_counts(), df_val['시제'].value_counts()

In [ ]:
df_train['확실성'].value_counts(), df_val['확실성'].value_counts()

In [741]:
df_train = pd.read_csv('df_train.csv')
df_val = pd.read_csv('df_val.csv')

In [ ]:
df_train

In [ ]:
df_val

## rs, rd

In [1050]:
import spacy
from spacy.lang.ko.examples import sentences 
nlp = spacy.load("ko_core_news_sm") #python -m spacy download ko_core_news_sm

### 불용어 가져오기

In [909]:
# open 불용어.txt, stop_words_set(불용어 사전) 만듬.
import locale

# 지역 설정 변경
locale.setlocale(locale.LC_COLLATE, 'ko_KR.UTF-8')

stop_words = {}

with open('불용어.txt', 'r') as f:
    for line in f:
        line = line.strip('\n')
        line = line.split(':')  # 줄 바꿈 문자 제거
        key = line[0] #key
        value = line[1].split(', ')  # value
        value.sort(key=locale.strxfrm)
        stop_words[key] = value  # 딕셔너리에 추가


# stop_words_set
stop_words_lst=[]
for _, value in stop_words.items():
    stop_words_lst.extend(value)

stop_words_set=set(stop_words_lst)
stop_words_set.update(nlp.Defaults.stop_words)

In [910]:
neg_adv_lst=['못', '안', '전혀', '결코', '별로', '그리', '도대체', '도저히', '절대', '그다지', '절대로', '도무지', '채', '미처', '좀처럼', '여간', '영', '비단', '차마', '통', '구태여']

In [911]:
for neg_adv in neg_adv_lst:
    if neg_adv in stop_words_set:
        stop_words_set.remove(neg_adv)

In [912]:
'수' in stop_words_set

True

In [913]:
stop_words_set.remove('수') #예측에 많이 쓰이는 의존명사(수) 제거

### random deletion

In [1051]:
def random_deletion(words, p):

    new_words=[]
    for token in words:
        r=random.uniform(0, 1)
        spt = token.lemma_.split('+')
        if r>p:
            if token.text in stop_words_set: #1-p의 확률로 불용어를 제거한다.
                continue
            if (token.pos_ =='ADV') and (token.dep_ not in ['ROOT', 'fixed', 'conj']) and (token.text not in neg_adv_lst): #1-p의 확률로 부정 부사를 제외한 부사 제거
                continue
            if token.pos_ in ['ADJ', 'NUM'] and token.dep_ not in ['ROOT', 'fixed', 'conj']: #1-p확률로 수식하는 단어를 제거
                continue
            if token.pos_ != 'VERB' and len(spt) == 2 and spt[0] != '나' and spt[-1] in ['은', '는', '이', '가', '을', '를', '에', '에게', '의', '께서']:
                new_words.append(''.join(spt[:-1]))
                continue
        
        new_words.append(token.text) #포함되는 토큰을 담는다.

    if len(new_words) < 4:
      new_words = [token.text for token in words]
      
    return new_words

In [1053]:
random_deletion(nlp('하늘을 나는 꼬마 자동차 붕붕을 봤어'), 0)

['하늘', '나는', '자동차', '붕붕', '봤어']

In [1055]:
doc = nlp('하늘을 나는 꼬마 자동차 붕붕을 봤어')
for token in doc:
    print(token.text, token.pos_, token.dep_, token.lemma_)

하늘을 NOUN obj 하늘+을
나는 PRON ccomp 나+는
꼬마 ADV compound 꼬마
자동차 NOUN compound 자동차
붕붕을 CCONJ ROOT 붕붕+을
봤어 PROPN conj 봤어


### random swap

In [974]:
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)

	return new_words

def swap_word(new_words):
	if len(new_words) > 4:
		random_idx_1 = random.randint(1, len(new_words)-2) #양 끝 단어는 교체하지 못하게 함.
		random_idx_2 = random_idx_1
		counter = 0

		while random_idx_2 == random_idx_1:
			random_idx_2 = random.randint(1, len(new_words)-2)
			counter += 1
			if counter > 3:
				return new_words

		new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words


### EDA(Easy Data Augmentation)

In [975]:
df_train = pd.read_csv('df_train.csv')
df_val = pd.read_csv('df_val.csv')

In [976]:
def EDA(sentence, alpha_rs=0.15, p_rd=0.5): 

	docs = nlp(sentence)
	words = random_deletion(docs, p_rd)
	
	if words:
		num_words=len(words)
		if num_words<20:
			n_rs = max(1, int(alpha_rs*num_words))
			a_words = random_swap(words, n_rs)
			augmented_sentences = " ".join(a_words)

		else:	augmented_sentences = " ".join(words)

	return augmented_sentences

In [977]:
def df_augmentation(df):

    df_aug = df.copy()

    for i in range(len(df)):
        
        r = random.uniform(0, 1)
        sentence_type = df['유형'][i]
        sentence_polarity = df['극성'][i]
        sentence_certainty = df['확실성'][i]
        sentence_tense = df['시제'][i]


        if sentence_type == '추론형':
            if r <= 0.5:
                df_aug['문장'][i] = EDA(df_aug['문장'][i])
                continue

        if sentence_type != '사실형' and sentence_type != '추론형':
            df_aug['문장'][i] = EDA(df_aug['문장'][i])
            continue

        
        if sentence_certainty == '불확실':
            df_aug['문장'][i] = EDA(df_aug['문장'][i])
            continue

        if sentence_polarity != '긍정':
            df_aug['문장'][i] = EDA(df_aug['문장'][i])
            continue

        if sentence_tense == '미래':
            if r <= 0.7:
                df_aug['문장'][i] = EDA(df_aug['문장'][i])
                continue

        else:
            if r<=0.3:
                df_aug['문장'][i] = EDA(df_aug['문장'][i])

    return df_aug



In [1033]:
df_train_aug_1 = df_augmentation(df_train)
df_train_aug_2 = df_augmentation(df_train)
df_train_aug_3 = df_augmentation(df_train)
df_train_aug_4 = df_augmentation(df_train)

df_train_aug = pd.concat([df_train_aug_1, df_train_aug_2, df_train_aug_3, df_train_aug_4, df_train], axis=0).drop_duplicates().reset_index(drop=True)

In [ ]:
df_train_aug

In [1039]:
df_val_aug_1 = df_augmentation(df_val)
df_val_aug_2 = df_augmentation(df_val)
df_val_aug_3 = df_augmentation(df_val)
df_val_aug_4 = df_augmentation(df_val)

df_val_aug = pd.concat([df_val_aug_1, df_val_aug_2, df_val_aug_3, df_val_aug_4, df_val], axis=0).drop_duplicates().reset_index(drop=True)

In [1003]:
len(df_train)

13425

In [1004]:
len(df_train_aug)

30708

In [1043]:
len(df_val_aug)

9357

#### value_counts

In [ ]:
df_train.유형.value_counts(), df_train_aug['유형'].value_counts()

In [ ]:
df_train.극성.value_counts(normalize=True), df_train_aug['극성'].value_counts(normalize=True)

In [ ]:
df_train.시제.value_counts(), df_train_aug['시제'].value_counts()

In [ ]:
df_train.확실성.value_counts(), df_train_aug['확실성'].value_counts()

In [ ]:
df_val_aug.유형.value_counts(), df_val.유형.value_counts()

In [ ]:
df_val_aug.극성.value_counts(), df_val.극성.value_counts()

In [ ]:
df_val_aug.시제.value_counts(), df_val.시제.value_counts()

In [ ]:
df_val_aug.확실성.value_counts(), df_val.확실성.value_counts()

In [ ]:
df_train_aug.isna().sum()

In [ ]:
df_val_aug.isna().sum()

In [1046]:
df_train_aug.to_csv('df_train_aug2.csv', index=False, encoding='utf-8-sig')

In [1047]:
df_val_aug.to_csv('df_val_aug2.csv', index=False, encoding='utf-8-sig')